In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from skimage import  io, feature

## Load file folder

In [ ]:
def get_filelist(directory):
    filelist = []

    for home, dirs, files in os.walk(directory):

        for file in files:
            filelist.append(os.path.join(home, file))

    return filelist

## Get image list and image name

In [ ]:
def get_imagelist(directory):
    filelist = []
    imgList = []
    imgName = []

    for home, dirs, files in os.walk(directory):

        for file in files:
            filelist.append(os.path.join(home, file))
            imgName.append(file)
    for i in range(len(filelist)):
        filetype = os.path.splitext(filelist[i])[-1]
        if filetype == '.txt':
            if i == 0:
                temp = np.zeros((200, 200))
            else:
                temp = np.zeros(np.shape(imgList[0]))
            txtfile = np.loadtxt(filelist[i]).astype(np.int64)
            for nx in range(np.shape(txtfile)[0]):
                for ny in range(np.shape(txtfile)[1]):
                    temp[nx][ny] = int(txtfile[nx][ny])
            imgList.append(temp)
        else:
            imgList.append(np.array(io.imread(filelist[i])))
    return imgList, imgName

## Load strain map folders and get image lists

In [ ]:
data_folder = './data_folder'
Folderlist = []
Foldername = []

for home, dirs, files in os.walk(data_folder):
    for dir in dirs:
        Folderlist.append(os.path.join(home, dir))
        Foldername.append(dir)
print(Folderlist)
print(Foldername)
# imgList [0] -> Mask, [1] -> diffraction image, [2] -> d-spacing strain map, [3] -> z-bending strain map, [4] -> clustering map. 
num_samples = len(Folderlist)
print(num_samples)

## Rocking curve angle of every LiCoO2 particles

In [ ]:
two_theta_list = [
    np.linspace(10.1, 12.1, 21),
    np.linspace(-27.4, -23.5, 40),
    np.linspace(-23.8, -19.8, 41),
    np.linspace(-27, -23, 41),
    np.linspace(-39.7, -37.7, 21),
    np.linspace(-4.7, -4.26, 23),
    np.linspace(-7.5, -5.5, 21),
    np.linspace(-22.55, -19.25, 34)
]

## Rocking curves of domain and boundary regions

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(24, 6), sharex=True)

for i in range(0, num_samples):
    imgList, imgName = get_imagelist(Folderlist[i])
    # get dim
    dim_z, dim_x, dim_y = imgList[1].shape
    # count cluster label number
    cluster_number = int(np.max(imgList[4]))
    label_number = cluster_number + 1
    roi_profile_list = [[] for i in range(cluster_number)]
    domain_boundary_profile_list = [[], []]
    
    # plot all average rocking curves of every domains
    cmap = cm.get_cmap("RdBu", cluster_number)
    colors = cmap(np.linspace(0, 1, cluster_number))
    
    for idx in range(1, label_number):
        label_roi = np.zeros_like(imgList[2])
        for nz in range(dim_z):
            label_roi[nz - 1][np.nonzero(imgList[4] == idx)] = imgList[1][nz - 1][np.nonzero(imgList[4] == idx)]
            avg_z = np.sum(label_roi[nz - 1]) / np.count_nonzero(imgList[4] == idx)
            roi_profile_list[idx - 1].append(avg_z)
        axs[i // 4, i % 4].plot(two_theta_list[i], roi_profile_list[idx - 1], color=colors[idx - 1])
        axs[i // 4, i % 4].set_yticks([])

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(24, 6), sharex=True)

for idx in range(1, label_number):
    domain_roi = np.zeros_like(imgList[2])
    boundary_roi = np.zeros_like(imgList[2])
    
    domain_mask = np.zeros_like(imgList[0])
    boundary_mask = np.zeros_like(imgList[0])
    
    # detect boundary region using canny
    domain_boundary = feature.canny(imgList[4], sigma=1)
    particle_boundary = feature.canny(imgList[0], sigma=0.1)
    
    for nx in range(np.shape(imgList[0])[0]):
        for ny in range(np.shape(imgList[0])[1]):
            if imgList[-1][nx][ny] != 0 and domain_boundary[nx][ny] == 0 and particle_boundary[nx][ny] == 0 and \
                    imgList[0][nx][ny] != 0:
                domain_mask[nx][ny] = 1
            if domain_boundary[nx][ny] != 0 and imgList[0][nx][ny] != 0 and imgList[-1][nx][ny] != 0:
                boundary_mask[nx][ny] = 1
                
    for nz in range(dim_z):
        domain_roi[nz - 1][np.nonzero(domain_mask == 1)] = imgList[2][nz - 1][np.nonzero(domain_mask == 1)]
        boundary_roi[nz - 1][np.nonzero(boundary_mask == 1)] = imgList[2][nz - 1][np.nonzero(boundary_mask == 1)]
        avg_z_domain = np.sum(domain_roi[nz - 1]) / np.count_nonzero(domain_mask != 0)
        avg_z_boundary = np.sum(boundary_roi[nz - 1]) / np.count_nonzero(boundary_mask != 0)
        domain_boundary_profile_list[0].append(avg_z_domain)
        domain_boundary_profile_list[1].append(avg_z_boundary)
        
    axs[i // 4, i % 4].plot(two_theta_list[i], domain_boundary_profile_list[0], label='domain', color='red')
    axs[i // 4, i % 4].plot(two_theta_list[i], domain_boundary_profile_list[1], label='boundary', color='blue')
    axs[i // 4, i % 4].legend(handlelength=1, frameon=False)
    axs[i // 4, i % 4].set_yticks([])
    
plt.show()

In [ ]:
boundary_strain = [[] for i in range(num_samples)]
domain_strain = [[] for i in range(num_samples)]

for i in range(num_samples):
    imgList, imgName = get_imagelist(Folderlist[i])
    contours = (feature.canny(imgList[4],sigma=1))^feature.canny(imgList[0],sigma=0.5)
    domain_boundary = feature.canny(imgList[4], sigma=1)
    particle_boundary = feature.canny(imgList[0], sigma=0.1)
    # d-spacing strain map statistics
    # j = 2
    j = 1
    for nx in range(np.shape(imgList[0])[0]):
        for ny in range(np.shape(imgList[0])[1]):
            if imgList[4][nx][ny] != 0 and domain_boundary[nx][ny]==0 and particle_boundary[nx][ny]==0 and imgList[0][nx][ny]!=0:
                domain_strain[i].append(imgList[j][nx][ny])
            if domain_boundary[nx][ny]!=0 and imgList[0][nx][ny]!=0 and imgList[4][nx][ny] !=0: 
                boundary_strain[i].append(imgList[j][nx][ny])

# calculate strain difference and std difference between domains and boundaries
boundary_strain_mean = [np.mean(boundary_strain[i]) for i in range(num_samples)]
domain_strain_mean = [np.mean(domain_strain[i]) for i in range(num_samples)]
strain_difference = np.array(domain_strain_mean)-np.array(boundary_strain_mean)
boundary_strain_std = [np.std(boundary_strain[i]) for i in range(num_samples)]
domain_strain_std = [np.std(domain_strain[i]) for i in range(num_samples)]
std_difference =  np.array(domain_strain_std)-np.array(boundary_strain_std)